In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_df = pd.read_csv('train.csv')
census = pd.read_csv('census.csv', index_col='cfips')
df_cleaned = pd.read_csv('df_cleaned.csv')
df = pd.merge(train_df, df_cleaned, on=['cfips', 'row_id'])
df = df.drop('Unnamed: 0', axis=1)
df = df.rename(columns={'< 30k pop': 'l_30k', '> 225k pop': 'g_225k'})

In [131]:
df

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,orders_rank,merchants_rank,gmv_rank,...,non_inst_pop,total_pop,pct_non_inst_pop,employed,pct_employed,unemployed,pct_unemployed,DGS10_last,tax_rate,tax_rate_diff
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,668.0,342.0,693.0,...,3935925,2270281,57.7,2204772,56.0,65509,2.9,2.02,6.03,0.89
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,511.0,550.0,396.0,...,3938622,2271806,57.7,2206223,56.0,65583,2.9,1.50,6.03,0.89
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,723.0,699.0,825.0,...,3941513,2273771,57.7,2207294,56.0,66477,2.9,1.68,6.03,0.89
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,624.0,491.0,571.0,...,3944209,2275680,57.7,2207568,56.0,68112,3.0,1.69,6.03,0.89
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,1023.0,777.0,1026.0,...,3946542,2276674,57.7,2206498,55.9,70176,3.1,1.78,6.03,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107970,56045_2022-02-01,56045,Weston County,Wyoming,2022-02-01,1.749688,98,NaN,NaN,NaN,...,455580,290102,63.7,279063,61.3,11039,3.8,1.79,0.00,0.00
107971,56045_2022-03-01,56045,Weston County,Wyoming,2022-03-01,1.767542,99,1198.0,1093.0,1351.0,...,455939,290282,63.7,279691,61.3,10591,3.6,1.83,0.00,0.00
107972,56045_2022-04-01,56045,Weston County,Wyoming,2022-04-01,1.767542,99,1308.0,1120.0,1306.0,...,456266,290379,63.6,280512,61.5,9867,3.4,2.32,0.00,0.00
107973,56045_2022-05-01,56045,Weston County,Wyoming,2022-05-01,1.803249,101,1300.0,1080.0,988.0,...,456588,290691,63.7,281100,61.6,9591,3.3,2.89,0.00,0.00


In [132]:
from tqdm import tqdm

Anomaly_Detection = True

if Anomaly_Detection:
    Anomaly_TH = 0.2
    outliers = []
    cnt = 0

    # Iterate through each unique cfips value
    for o in tqdm(df.cfips.unique()):
        indices = (df['cfips'] == o)
        # Get all the rows for the current cfips and reset the index
        tmp = df.loc[indices].copy().reset_index(drop=True)
        var = tmp.microbusiness_density.values.copy()
        # Iterate through the density values in reverse order
        # starting from the second last index
        for i in range(33, 1, -1):
            # Calculate the anomaly threshold as 20% of the average
            # of the values before the current index
            thr = Anomaly_TH * np.mean(var[:i])

            diff = abs(var[i] - var[i - 1])
            if diff >= thr:
                # Adjust all the previous values to the same 'stage' as the current point
                var[:i] *= (var[i] / var[i - 1])
                # Save cfips that have outliers
                outliers.append(o)
                cnt += 1

        # Adjust the first value to be almost the same as the second value
        var[0] = var[1] * 0.99
        df.loc[indices, 'microbusiness_density'] = var

    # Get the unique cfips values that have outliers
    outliers = np.unique(outliers)
    print(f'------------------------------')
    print(f'there are {len(outliers)} county have {cnt} outliners in total.')
    print(f'------------------------------')


 39%|███▉      | 1213/3085 [00:00<00:00, 2074.36it/s]/var/folders/bb/g8c6d14n37lb8f846cg2t1_m0000gn/T/ipykernel_23133/3585463504.py:26: RuntimeWarning: invalid value encountered in double_scalars
  var[:i] *= (var[i] / var[i - 1])
100%|██████████| 3085/3085 [00:01<00:00, 2057.94it/s]

------------------------------
there are 447 county have 673 outliners in total.
------------------------------


In [133]:
state_dict = df[['cfips', 'state', 'county']]
state_dict = state_dict.set_index('cfips')
state_dict = state_dict.drop_duplicates()
state_dict = state_dict.to_dict()

zero_mask = (df['microbusiness_density'] == 0) & (df['active'] == 0) 
df['ratio'] = df.loc[~ zero_mask].groupby('cfips', group_keys=False).apply(lambda x: x.active / x.microbusiness_density)
df['ratio'] = df['ratio'].fillna(0)
df.state = df['state'].astype('category').cat.codes
df = df.drop(columns=['county'])

In [134]:
df.first_day_of_month = pd.to_datetime(df.first_day_of_month)
df['month'] = df.first_day_of_month.dt.month
df['year'] = df.first_day_of_month.dt.year - 2019
df['is_new_year'] = 0
df.loc[df.month == 1, 'is_new_year'] = 1
seasons = {1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'}
df['season'] = df['month'].apply(lambda x: seasons[x])
df['md_national_avg'] = df.groupby(['year','month'])['microbusiness_density'].transform('mean')
df['md_state_avg'] = df.groupby(['state','year','month'])['microbusiness_density'].transform('mean')
df['a_national_avg'] = df.groupby(['year','month'])['active'].transform('mean')
df['a_state_avg'] = df.groupby(['state','year','month'])['active'].transform('mean')


In [135]:
df = pd.concat([df, pd.get_dummies(df['season'], prefix='season')], axis=1)
df = df.drop(columns='season')

In [136]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

# Create a US holiday calendar
cal = USFederalHolidayCalendar()

holidays = cal.holidays(start='2019-01-01', end='2023-06-01')
df['holidays'] = pd.to_datetime(df['first_day_of_month'], format='%Y-%m-%d').dt.to_period('M').apply(lambda x: len(holidays[holidays.to_period('M') == x]))

In [137]:
df['idx'] = df.groupby('cfips')['cfips'].cumcount()
df = df.merge(census, on='cfips')

In [138]:
import numpy as np

def calculate_log_diff(df, groupby_col, column, mask, new_col_name):
    df[new_col_name] = df.loc[~mask].groupby(groupby_col, group_keys=False)[column].apply(lambda x: np.log(x).diff())
    df.loc[mask, [new_col_name]] = 0
    df[new_col_name] = df.loc[~mask].groupby(groupby_col, group_keys=False)[new_col_name].bfill()
    return df

def create_lags(df, groupby_col, columns, lags):
    for column in columns:
        for lag in lags:
            new_col_name = f'{column}_lag_{lag}'
            df[new_col_name] = df.groupby(groupby_col, group_keys=False)[column].shift(lag)
    return df

def create_target(df, groupby_col, column, mask, new_col_name):
    df[new_col_name] = df.loc[~mask].groupby(groupby_col, group_keys=False)[column].apply(lambda x: np.log(x.shift(-1)) - np.log(x))
    df.loc[mask, [new_col_name]] = 0
    return df

def fill_target_with_train(df_test, train_df):
    df_test = df_test.merge(train_df[['target1']], how='left', left_index=True, right_index=True)
    df_test['target'] = df_test['target'].fillna(df_test['target1'])
    df_test = df_test.drop(columns='target1')
    return df_test

# Preprocessing
df = df.set_index('row_id')
mask = ((df.microbusiness_density == 0) & (df.active == 0))

# Calculate log differences
df = calculate_log_diff(df, 'cfips', 'md_state_avg', mask, 'md_state_avg_log_diff')
df = calculate_log_diff(df, 'cfips', 'a_state_avg', mask, 'a_state_avg_log_diff')

# Create lags
LAGS = [1, 2, 3, 4, 5, 6]
columns_to_lag = ['microbusiness_density', 'active']
df = create_lags(df, 'cfips', columns_to_lag, LAGS)

# Create target
df = create_target(df, 'cfips', 'microbusiness_density', mask, 'target')

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [139]:
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

def get_NN_cfips_by_feats(df, N, p, metric, feature):
    """
    Get nearest neighbors based on a specific feature.
    """
    df = df.reset_index()
    df = df[['cfips', 'first_day_of_month', feature]]
    df = df.pivot(index='cfips', columns='first_day_of_month', values=feature)
    df.columns.name = None
    df = df.fillna(0)
    nn = NearestNeighbors(n_neighbors=N, p=p, metric=metric)
    nn.fit(df)
    neighbors = nn.kneighbors(df, return_distance=False)
    df_nn = pd.DataFrame(neighbors, index=df.index)
    df_nn = df_nn.apply(lambda col: df.index[col])
    df_nn = df_nn.iloc[:, 1:]
    return df_nn

def get_NN_cfips_by_census(census, N, p, metric):
    """
    Get nearest neighbors based on census data.
    """
    nn_census = NearestNeighbors(n_neighbors=N, p=p, metric=metric)
    census_na = census[census.isna().any(axis=1)]
    na_mask = census_na.index
    census.loc[na_mask] = census.loc[na_mask].fillna(method='ffill', axis=1)
    df_mask = df.cfips.unique()
    census = census.loc[df_mask]
    nn_census.fit(census)
    neighbors = nn_census.kneighbors(census, return_distance=False)
    df_nn = pd.DataFrame(neighbors, index=census.index)
    df_nn = df_nn.iloc[:, 1:]
    df_nn = df_nn.loc[df_mask]
    df_nn = df_nn.rename(columns={1: 'NN_1', 2: 'NN_2', 3: 'NN_3'})
    df_nn[['NN_1', 'NN_2', 'NN_3']] = df_nn[['NN_1', 'NN_2', 'NN_3']].applymap(lambda x: census.index[x])
    return df_nn

def generate_feature(df_nn, df, feature, colname):
    """
    Generate feature for nearest neighbors.
    """
    cfips = df.cfips.unique()
    df_grouped = df.groupby('cfips')
    dfs = []
    for cfips_val in tqdm(cfips):
        nn_list = df_nn.loc[cfips_val].values.tolist()
        feats = pd.DataFrame(np.asarray([df_grouped.get_group(i)[feature].values for i in nn_list]).T, columns=[f'{colname}_NN_{i}' for i in range(len(nn_list))])
        dfs.append(feats)
    result = pd.concat(dfs, axis=0)
    result = result.reset_index(drop=True)
    return result

def combine_nn_feats(df, df_nn):
    """
    Combine features with the original dataframe.
    """
    df = df.reset_index()
    df = pd.concat([df, df_nn], axis=1)
    df = df.drop(columns='first_day_of_month')
    df = df.set_index('row_id')
    return df


def generate_lagged_features(df_nn, df, feature, prefix):
    feature_dfs = []
    for lag in range(1, 6):  # Generate lags up to 5
        colname = f"{prefix}_lag_{lag}"
        feature_df = generate_feature(df_nn, df, f"{feature}_lag_{lag}", colname)
        feature_dfs.append(feature_df)
    return feature_dfs

KNN = True

if KNN:
    # generate neighbor
    df_nn_c = get_NN_cfips_by_census(census, N=4, p=1, metric='manhattan')
    df_nn_md = get_NN_cfips_by_feats(df, N=4, p=1, metric='manhattan', feature='microbusiness_density')
    df_nn_md_log_diff = get_NN_cfips_by_feats(df, N=4, p=1, metric='manhattan', feature='target')

    # generate features
    md_state_avg_diff_features = [
        generate_feature(df_nn_c, df, 'md_state_avg_log_diff', 'md_state_avg_c'),
        generate_feature(df_nn_md, df, 'md_state_avg_log_diff', 'md_state_avg_1'),
        generate_feature(df_nn_md_log_diff, df, 'md_state_avg_log_diff', 'md_state_avg_ld'),
    ]

    md_lag_features_c = generate_lagged_features(df_nn_c, df, 'microbusiness_density', 'md_c')
    md_lag_features_md = generate_lagged_features(df_nn_md, df, 'microbusiness_density', 'md_1')
    md_lag_features_md_log_diff = generate_lagged_features(df_nn_md_log_diff, df, 'microbusiness_density', 'md_ld')

    # Combine generated features
    NN_feats_list = md_state_avg_diff_features + md_lag_features_c + md_lag_features_md + md_lag_features_md_log_diff
    NN_feats = pd.concat(NN_feats_list, axis=1)
    
    # Merge the features with the training dataset
    df = combine_nn_feats(df, NN_feats)

else:
    df = df.drop(columns='first_day_of_month')


100%|██████████| 3085/3085 [00:01<00:00, 2183.48it/s]


In [140]:
mask = ((train_df.microbusiness_density == 0) & (train_df.active == 0))
train_df = create_target(train_df, 'cfips', 'microbusiness_density', mask, 'target1')
train_df = train_df.set_index('row_id')
df_train = df[~df.index.str.contains('2022-06-01')]
df_test = df[df.index.str.contains('2022-06-01')]
df_train = df_train.replace([np.inf, -np.inf], 0)
df_test = fill_target_with_train(df_test, train_df)

df_train = df_train.fillna(0)
target_train = df_train.target
df_train.drop(columns='target', inplace=True)

df_test = df_test.fillna(0)
target_test = df_test.target
df_test.drop(columns='target', inplace=True)

In [141]:
df_train.to_csv('df_train.csv')
df_test.to_csv('df_test.csv')
target_train.to_csv('target_train.csv')
target_test.to_csv('target_test.csv')

In [145]:
print(df_train.columns.tolist())

['cfips', 'state', 'microbusiness_density', 'active', 'orders_rank', 'merchants_rank', 'gmv_rank', 'avg_traffic', 'avg_lifespan_mths', '30k - 225k pop', 'l_30k', 'g_225k', 'nrc_order', 'nrc_merch', 'nrc_gmv', 'confirmed', 'deaths', 'people_vaccinated', 'people_fully_vaccinated', 'school_closing', 'workplace_closing', 'cancel_events', 'gatherings_restrictions', 'transport_closing', 'stay_home_restrictions', 'internal_movement_restrictions', 'international_movement_restrictions', 'information_campaigns', 'testing_policy', 'contact_tracing', 'facial_coverings', 'vaccination_policy', 'elderly_people_protection', 'government_response_index', 'stringency_index', 'containment_health_index', 'economic_support_index', 'ESTIMATESBASE2020', 'POPESTIMATE2020', 'POPESTIMATE2021', 'NPOPCHG2020', 'NPOPCHG2021', 'BIRTHS2020', 'BIRTHS2021', 'DEATHS2020', 'DEATHS2021', 'NATURALCHG2020', 'NATURALCHG2021', 'INTERNATIONALMIG2020', 'INTERNATIONALMIG2021', 'DOMESTICMIG2020', 'DOMESTICMIG2021', 'NETMIG2020', 

In [175]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(12, 6))  # Set figure size

ax1.plot(df_train.DGS10_last, label='DGS10_last', color='blue')
ax1.set_ylabel('Confirmed Cases', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()  # Create a second y-axis on the same x-axis
ax2.plot(df_train.microbusiness_density, label='Microbusiness Density', color='green')
ax2.set_ylabel('Microbusiness Density', color='green')
ax2.tick_params(axis='y', labelcolor='green')

ax1.set_xlabel('Date')  # Set x-axis label
ax1.legend(loc='upper left')  # Add legend for the first dataset
ax2.legend(loc='upper right')  # Add legend for the second dataset

plt.title('Confirmed Cases and Microbusiness Density')  # Add title
plt.show()


KeyboardInterrupt: 

In [170]:
df_train[df_train.cfips == 1005].DGS10_last

row_id
1005_2019-08-01    2.02
1005_2019-09-01    1.50
1005_2019-10-01    1.68
1005_2019-11-01    1.69
1005_2019-12-01    1.78
1005_2020-01-01    1.92
1005_2020-02-01    1.51
1005_2020-03-01    1.13
1005_2020-04-01    0.70
1005_2020-05-01    0.64
1005_2020-06-01    0.65
1005_2020-07-01    0.66
1005_2020-08-01    0.55
1005_2020-09-01    0.72
1005_2020-10-01    0.69
1005_2020-11-01    0.88
1005_2020-12-01    0.84
1005_2021-01-01    0.93
1005_2021-02-01    1.11
1005_2021-03-01    1.44
1005_2021-04-01    1.74
1005_2021-05-01    1.65
1005_2021-06-01    1.58
1005_2021-07-01    1.45
1005_2021-08-01    1.24
1005_2021-09-01    1.30
1005_2021-10-01    1.52
1005_2021-11-01    1.55
1005_2021-12-01    1.43
1005_2022-01-01    1.52
1005_2022-02-01    1.79
1005_2022-03-01    1.83
1005_2022-04-01    2.32
1005_2022-05-01    2.89
Name: DGS10_last, dtype: float64